**Import needed packages/modules**

In [ ]:
# Cell 1
from dataclasses import dataclass

import matplotlib.pyplot as plt
import numpy as np

**Define a `dataclass` to store a transformation mapping**\
Each transform requires <u>three</u> (2D) Cartesian point "mappings":
* The $1^{st}$ mapping relocates the $(bottom_{\ left})$ corner of the base frame
* The $2^{nd}$ mapping relocates the $(bottom_{\ right})$ corner of the base frame
* The $3^{rd}$ mapping relocates the $(top_{\ left})$ corner of the base frame

In [ ]:
# Cell 2
@dataclass
class Transform:
    def __init__(self):
        self.x1 = 0.0
        self.y1 = 0.0
        self.x2 = 0.0
        self.y2 = 0.0
        self.x3 = 0.0
        self.y3 = 0.0
        self.color = (0, 0, 0)  # Black in RGB
        self.probability = 0.0
        self.m = np.zeros((3, 3))

**Define a `dataclass` to hold a complete Iterated Function System**\
Each IFS contains:
1. The 2D Cartesian coordinates of the **base frame** for this IFS
2. A set of transforms
3. The  **probability** of each transform being applied in a given iteration
4. The  **pixel color** of each transform
5. A function that given an initial coordinate, will move it to a new coordinate based upon one of its randomly selected transforms

In [ ]:
# Cell 3
@dataclass
class IteratedFunctionSystem:
    def __init__(self):
        self.transforms = []
        self.affine_width = 0.0
        self.affine_height = 0.0
        self.cdf = 0.0

    def set_base_frame(self, x_min, y_min, x_max, y_max):
        self.affine_width = x_max - x_min
        self.affine_height = y_max - y_min

    # fmt: off
    def add_mapping(self, x_left, y_left, x_right, y_right,
                     x_top, y_top, color, probability):
    #fmt: on
        # Probabilities accumulate across mappings
        self.cdf += probability

        t = Transform()
        t.x1 = x_left
        t.y1 = y_left
        t.x2 = x_right
        t.y2 = y_right
        t.x3 = x_top
        t.y3 = y_top
        t.color = color
        t.probability = self.cdf
        self.transforms.append(t)

    def generate_transforms(self):
        for t in self.transforms:
            coeffs = np.array([
                [0, 0, 1],
                [self.affine_width, 0, 1],
                [0, self.affine_height, 1],
            ])

            # Solve systems of 3x3 equations to get transformation matrix
            vals = np.array([t.x1, t.x2, t.x3])
            sol = np.linalg.solve(coeffs, vals)
            t.m[0, 0] = sol[0]
            t.m[1, 0] = sol[1]
            t.m[2, 0] = sol[2]

            vals = np.array([t.y1, t.y2, t.y3])
            sol = np.linalg.solve(coeffs, vals)
            t.m[0, 1] = sol[0]
            t.m[1, 1] = sol[1]
            t.m[2, 1] = sol[2]

            # Last column in transformation matrix is always this
            t.m[0, 2] = 0
            t.m[1, 2] = 0
            t.m[2, 2] = 1

    def transform_point(self, x, y):
        p = np.random.random()
        for t in self.transforms:
            if p <= t.probability:
                xt = x * t.m[0, 0] + y * t.m[1, 0] + t.m[2, 0]
                yt = x * t.m[0, 1] + y * t.m[1, 1] + t.m[2, 1]
                return xt, yt, t.color
        # We should never get here if the
        # mapping probabilities all sum to 1.0
        return 0, 0, (0,0,0)

**Define a function to apply (200,000 iterations) the IFS to an initial $(0,0)$ starting point**

In [ ]:
# Cell 4
def calc_ifs(scr, world_rect, ifs):
    # Array shapes are ROW x COL, so the x size is the 2nd element
    # Also, we subtract one because array indexes start at 0
    screen_x_max = scr.shape[1] - 1
    screen_y_max = scr.shape[0] - 1

    world_x_min = world_rect[0][0]
    world_y_min = world_rect[0][1]
    world_x_max = world_rect[1][0]
    world_y_max = world_rect[1][1]
    world_x_size = world_x_max - world_x_min
    world_y_size = world_y_max - world_y_min

    x, y = 0.0, 0.0

    # Iterate 100 times, but don't draw any pixels
    # This will give the IFS "time" reach its stable orbit
    for _ in range(100):
        x, y, clr = ifs.transform_point(x, y)

    # Now draw each pixel in the stable orbit
    for _ in range(200_000):
        x, y, clr = ifs.transform_point(x, y)
        sx = int(screen_x_max * (x - world_x_min) / world_x_size)
        sy = int(screen_y_max - screen_y_max * (y - world_y_min) / world_y_size)
        if 0 <= sx <= screen_x_max and 0 <= sy <= screen_y_max:
            scr[sy, sx] = clr

**Given a `world_rect`, create and draw the [Barnsley fern](https://en.wikipedia.org/wiki/Barnsley_fern) IFS**


In [ ]:
# Cell 5
def draw_ifs(world_rect):
    # RGB triplets (one tuple for each color)
    red, green, blue = (255, 0, 0), (0, 255, 0), (0, 0, 255)
    yellow, white = (255, 255, 0), (255, 255, 255)
    orange, yellow = (255, 165, 0), (255, 255, 0)
    lightblue, sienna, papayawhip = (173, 216, 230), (160, 82, 45), (255, 239, 213)
    purple, pink = (128, 0, 128), (255, 192, 203)
    chartreuse, maroon = (127, 255, 0), (128, 0, 0)

    p = 1 / 29

    ifs = IteratedFunctionSystem()
    ifs.set_base_frame(0, 0, 17, 7)
    ifs.add_mapping(0, 6, 2, 6, 0, 7, lightblue, p)  # 01
    ifs.add_mapping(2, 6, 4, 6, 2, 7, blue, p)  # 02
    ifs.add_mapping(1, 4, 1, 6, 0, 4, blue, p)  # 03
    ifs.add_mapping(4, 6, 4, 4, 5, 6, blue, p)  # 04
    ifs.add_mapping(0, 3, 2, 3, 0, 4, blue, p)  # 05
    ifs.add_mapping(4, 4, 2, 4, 4, 3, red, p)  # 06
    ifs.add_mapping(1, 1, 1, 3, 0, 1, pink, p)  # 07
    ifs.add_mapping(4, 3, 4, 1, 5, 3, blue, p)  # 08
    ifs.add_mapping(0, 0, 2, 0, 0, 1, blue, p)  # 09
    ifs.add_mapping(4, 1, 2, 1, 4, 0, sienna, p)  # 10
    ifs.add_mapping(6, 6, 7, 6, 6, 7, blue, p)  # 11
    ifs.add_mapping(10, 6, 11, 6, 10, 7, papayawhip, p)  # 12
    ifs.add_mapping(6, 5, 8, 5, 6, 6, chartreuse, p)  # 13
    ifs.add_mapping(11, 4, 11, 6, 10, 4, blue, p)  # 14
    ifs.add_mapping(7, 3, 7, 5, 6, 3, blue, p)  # 15
    ifs.add_mapping(8, 4, 9, 4, 8, 5, blue, p)  # 16
    ifs.add_mapping(7, 1, 7, 3, 6, 1, orange, p)  # 17
    ifs.add_mapping(9, 2, 9, 4, 8, 2, blue, p)  # 18
    ifs.add_mapping(10, 4, 10, 2, 11, 4, maroon, p)  # 19
    ifs.add_mapping(6, 0, 7, 0, 6, 1, blue, p)  # 20
    ifs.add_mapping(9, 1, 11, 1, 9, 2, blue, p)  # 21
    ifs.add_mapping(10, 0, 11, 0, 10, 1, yellow, p)  # 22
    ifs.add_mapping(13, 5, 13, 7, 12, 5, green, p)  # 23
    ifs.add_mapping(12, 4, 13, 4, 12, 5, blue, p)  # 24
    ifs.add_mapping(12, 4, 12, 2, 13, 4, blue, p)  # 25
    ifs.add_mapping(13, 1, 13, 2, 12, 1, white, p)  # 26
    ifs.add_mapping(12, 0, 14, 0, 12, 1, blue, p)  # 27
    ifs.add_mapping(15, 0, 15, 1, 14, 0, blue, p)  # 28
    ifs.add_mapping(15, 0, 17, 0, 15, 1, purple, p)  # 29
    ifs.generate_transforms()

    scr = np.zeros((500, 1200, 3), dtype=np.uint8)
    calc_ifs(scr, world_rect, ifs)
    plt.figure(figsize=(17, 7))
    plt.imshow(scr, interpolation="nearest", aspect="equal")
    plt.axis("off")
    plt.show()

**Draw the BNL IFS from $(0,0)$ to $(17,7)$**

In [ ]:
# Cell 6
draw_ifs(((0, 0), (17, 7)))

**Draw the BNL IFS from $(5.7281,4.8537)$ to $(8.8899,6.1583)$**

In [ ]:
# Cell 7
draw_ifs(((5.7281, 4.8537), (8.8899, 6.1583)))